In [168]:
import pandas as pd
import numpy as np

import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import f1_score

from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /Users/en-
[nltk_data]     ning.chiang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/en-
[nltk_data]     ning.chiang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/en-
[nltk_data]     ning.chiang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/en-
[nltk_data]     ning.chiang/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [169]:
df = pd.read_csv('podcast_sample_new.csv', lineterminator='\n', index_col = 0)
df

,podcast_id,title,content,rating,author_id,created_at,itunes_id,slug,itunes_url,podcast_title,category
0,b313ef8ef0d5b64290d3036ff1bbf2d2,감성 라디오 음악도시,미국 서부에 있는 유학생이에요. 성시경씨 제대 후 라디오 복귀만 기다려오다가 6 월...,5,664CCA7142E9AE8,2011-09-14T13:25:46-07:00,442838670,fm-%EC%9D%8C%EC%95%85%EB%8F%84%EC%8B%9C-%EC%A2...,https://podcasts.apple.com/us/podcast/fm-%EC%9...,FM 음악도시(종영),music
1,abfb842993be20d21bfae7103addc5e9,They’ve really cut back on the content this se...,Last season there was a new pod every 3-4 days...,1,AD790CE113DCBC1,2018-04-11T13:46:47-07:00,1015394113,the-good-phight-for-philadelphia-phillies-fans,https://podcasts.apple.com/us/podcast/the-good...,The Good Phight: for Philadelphia Phillies fans,sports
2,ebdf879a424547d01862a9bbba18a0f3,Good info. source...,Bob brings a lot of knowledge to any firearm d...,4,E223A4B2642C970,2010-01-19T08:11:43-07:00,333180229,handgun-world-podcast,https://podcasts.apple.com/us/podcast/handgun-...,Handgun World Podcast,news
3,ab2fdb7db023b223d870487165d11ff3,Mixed,They have lost much of thier credibility by de...,3,E1E7DBE750D119E,2021-01-28T12:21:49-07:00,971901464,wsj-opinion-potomac-watch,https://podcasts.apple.com/us/podcast/wsj-opin...,WSJ Opinion: Potomac Watch,news
4,ca601bd1524322d0527b16adf2738ff3,Try it now!,Even better than I expected. I was interested ...,5,D7CA4858AFA2CFC,2017-08-24T10:55:20-07:00,1257821731,conversations-with-people-who-hate-me,https://podcasts.apple.com/us/podcast/conversa...,Conversations with People Who Hate Me,society
...,...,...,...,...,...,...,...,...,...,...,...
49995,a0f4bdf8c437e34e5b6a17698fbdf64d,🔥Sussy🔥,"A little bit $u$, make an episode on plant pol...",2,45C72657925F18D,2022-03-28T07:16:21-07:00,1103320303,but-why-a-podcast-for-curious-kids,https://podcasts.apple.com/us/podcast/but-why-...,But Why: A Podcast for Curious Kids,kids
49996,d5683ccc88341197ca42aad82111c96f,Jumping butterballs!!,"If you are a Marx Brothers fan, and have been ...",5,AAF30F4EAE42219,2018-05-28T18:10:19-07:00,1363963113,the-marx-brothers-council-podcast,https://podcasts.apple.com/us/podcast/the-marx...,The Marx Brothers Council Podcast,tv
49997,cc546886c2be6370d76bd638f5256ba6,Way too much junkie commercials,I love the show very interesting I’m telling t...,1,2032AE3B4B98F1F,2020-05-20T17:01:07-07:00,1484888427,autopsy-the-last-hours-of,https://podcasts.apple.com/us/podcast/autopsy-...,Autopsy: The Last Hours Of…,society
49998,aaeb50f39da1a9a1d2e3837cf4edb2d0,Awful,Kelly Havens is not inspirational. Her husband...,1,ED81029C263B8BA,2022-01-15T13:36:52-07:00,1494284839,simple-farmhouse-life,https://podcasts.apple.com/us/podcast/simple-f...,Simple Farmhouse Life,leisure


In [193]:
df.groupby('category')['podcast_id'].agg({'count'})

,count
category,
arts,3181
business,3783
comedy,8019
crime,2521
education,2988
fiction,776
government,413
health,2353
history,1417


In [170]:
neg = df[df['rating']<4]
pos = df[df['rating']>=4]

### positive 

In [4]:
vectorizer = TfidfVectorizer(ngram_range=(2,2),
                             token_pattern=r'\b[a-zA-Z]{3,}\b',
                             max_df=0.4, max_features=1000, stop_words=stopwords.words())

corpus = list(pos["content"].values)

X = vectorizer.fit_transform(corpus)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)

/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aldiz', 'baiknya', 'baizik', 'berkali', 'bukatzeko', 'edota', 'eze', 'ezpabere', 'ezpada', 'ezperen', 'gainera', 'gainerontzean', 'guztiz', 'hainbestez', 'horra', 'kali', 'kurangnya', 'mata', 'olah', 'ordea', 'osterantzean', 'printr', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(
/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [5]:
tf_idf = tf_idf.sum(axis=1)
score = pd.DataFrame(tf_idf, columns=["score"])
score.sort_values(by="score", ascending=False, inplace=True)

In [6]:
score.head(30)

,score
love podcast,758.174728
great podcast,445.930163
love show,299.765015
listening podcast,270.720708
love listening,265.052220
great job,257.819878
podcast great,252.988045
great work,249.873717
highly recommend,236.912586
great show,208.879908


In [7]:
REMOVE_STOP_WORDS = True
STEMMING = False
LEMMATIZE = True
VECTORIZER = 'TFIDF' # TFIDF or COUNT
NGRAM_SIZE = (2,2) # (1,1), (2,2), or (3,3)

In [8]:
# prep nltk
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

In [154]:
def cleantxt(cleanr):
    cleanr = cleanr.strip().lower()
    cleanr = re.sub(r'(podcasts?|listen|just|make|really|feel|need|don|love|year|week|great|job|work|pod|love love)\b',' ', cleanr)
#     cleanr = re.sub(r'\b(love|like|great|good|really|really enojoy|true crime|started listening|listen|listening|new episode)\b',' ', cleanr)
#     cleanr = re.sub(r'\b(look forward|looking forward|great work|great job|good job|good work|highly recommend)+(s?|es?)\b',' ', cleanr)
#     cleanr = re.sub(r'\b(great work|great job|look forward|looking forward|good job|good work|great show|listened episode|feel like)\b',' ', cleanr)
#     cleanr = re.sub(r'\b(favorite|absolutely|great|highly recommend|new|episode|love|enjoy|listen|listening|hard work|true crime)\b',' ', cleanr)
    
    word_tokens = word_tokenize(cleanr)
    
    if REMOVE_STOP_WORDS == True:
        filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        word_tokens = filtered_sentence
    
    if STEMMING == True:
        word_tokens = [ps.stem(w) for w in word_tokens]
        
    if LEMMATIZE == True:
        word_tokens = [lemmatizer.lemmatize(w) for w in word_tokens]
        
    output_text = ' '.join(word_tokens)
    return output_text

In [155]:
pos['clean_content'] = pos.apply(lambda x: cleantxt(x['content']), axis=1)

/var/folders/zy/z2gfj07x2s187jy3_c87nhj80000gq/T/ipykernel_2055/2783385914.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos['clean_content'] = pos.apply(lambda x: cleantxt(x['content']), axis=1)


In [156]:
# score again 
vectorizer = TfidfVectorizer(ngram_range=(2,2),
                             token_pattern=r'\b[a-zA-Z]{3,}\b',
                             max_df=0.4, max_features=1000, stop_words=stopwords.words())

corpus = list(pos["clean_content"].values)

X = vectorizer.fit_transform(corpus)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)
tf_idf = tf_idf.sum(axis=1)
score = pd.DataFrame(tf_idf, columns=["score"])
score.sort_values(by="score", ascending=False, inplace=True)
score.head(30)

/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aldiz', 'baiknya', 'baizik', 'berkali', 'bukatzeko', 'edota', 'eze', 'ezpabere', 'ezpada', 'ezperen', 'gainera', 'gainerontzean', 'guztiz', 'hainbestez', 'horra', 'kali', 'kurangnya', 'mata', 'olah', 'ordea', 'osterantzean', 'printr', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(
/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,score
highly recommend,285.784880
started listening,191.314029
forward episode,177.043222
listened episode,162.492897
enjoy listening,134.594290
wait hear,131.401650
long time,128.810358
wait episode,108.775226
listening episode,107.998922
real life,99.400959


In [157]:
vectorizer1 = TfidfVectorizer(ngram_range=(2,2),
                             min_df=0.01, max_df=0.6, stop_words="english")


X_review1, review_terms1 = vectorizer1.fit_transform(pos.clean_content), vectorizer1.get_feature_names_out()
review_tf_idf1 = pd.DataFrame(X_review1.toarray(), columns=review_terms1)
print(f"Podcast Review TF-IDF: {review_tf_idf1.shape}")
review_tf_idf1.head(5)

Podcast Review TF-IDF: (28249, 6)


,highly recommend,look forward,looking forward,new episode,started listening,true crime
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0


In [158]:
nmf1 = NMF(n_components=3)
W_review1 = nmf1.fit_transform(X_review1)
H_review1 = nmf1.components_
print(f"Original shape of X review is {X_review1.shape}")
print(f"Decomposed W review matrix is {W_review1.shape}")
print(f"Decomposed H review matrix is {H_review1.shape}")

Original shape of X review is (28249, 6)
Decomposed W review matrix is (28249, 3)
Decomposed H review matrix is (3, 6)


/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


In [159]:
from typing import List

def get_top_tf_idf_tokens_for_topic(H: np.array, feature_names: List[str], num_top_tokens: int = 5):
  """
  Uses the H matrix (K components x M original features) to identify for each
  topic the most frequent tokens.
  """
  for topic, vector in enumerate(H):
    print(f"TOPIC {topic}\n")
    total = vector.sum()
    top_scores = vector.argsort()[::-1][:num_top_tokens]
    token_names = list(map(lambda idx: feature_names[idx], top_scores))
    strengths = list(map(lambda idx: vector[idx] / total, top_scores))
    
    for strength, token_name in zip(strengths, token_names):
          print(f"\b{token_name} ({round(strength * 100, 1)}%)\n")
    print(f"=" * 50)

In [160]:
get_top_tf_idf_tokens_for_topic(H_review1, review_tf_idf1.columns.tolist(), 5)
print(f"Podcast Review Topics:\n\n")

TOPIC 0

look forward (87.0%)

new episode (10.7%)

started listening (1.2%)

true crime (1.1%)

looking forward (0.0%)

TOPIC 1

highly recommend (95.4%)

started listening (4.0%)

true crime (0.5%)

new episode (0.1%)

looking forward (0.0%)

TOPIC 2

looking forward (90.7%)

new episode (7.2%)

started listening (1.8%)

true crime (0.2%)

look forward (0.0%)

Podcast Review Topics:




### negative

In [108]:
vectorizer = TfidfVectorizer(ngram_range=(2,2),
                             token_pattern=r'\b[a-zA-Z]{3,}\b',
                             max_df=0.4, max_features=1000, stop_words=stopwords.words())

corpus = list(neg["content"].values)

X = vectorizer.fit_transform(corpus)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)

/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aldiz', 'baiknya', 'baizik', 'berkali', 'bukatzeko', 'edota', 'eze', 'ezpabere', 'ezpada', 'ezperen', 'gainera', 'gainerontzean', 'guztiz', 'hainbestez', 'horra', 'kali', 'kurangnya', 'mata', 'olah', 'ordea', 'osterantzean', 'printr', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(
/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [109]:
tf_idf = tf_idf.sum(axis=1)
score = pd.DataFrame(tf_idf, columns=["score"])
score.sort_values(by="score", ascending=False, inplace=True)

In [110]:
score.head(3)

,score
waste time,221.780865
love podcast,211.911576
hard listen,155.024631


In [ ]:
def word_replace(line):
    line = re.sub(r'\b(podcasts?|listen|just|really|don|like|lot|use|want|say|episode)\b', '', line)
    line = re.sub(r'\b(make|think|minute|love|pods?|good|year)\b', '', line)
    return line

In [146]:
def cleantxt(cleanr):
    cleanr = cleanr.strip().lower()
    cleanr = re.sub(r'\b(podcasts?|listen|just|really|don|like|lot|use|want|say|episode)\b',' ', cleanr)
    cleanr = re.sub(r'\b(make|think|minute|love|pods?|good|year)\b',' ', cleanr)
#     cleanr = re.sub(r'\b(love|like|great|good|really|really enojoy|true crime|started listening|listen|listening|new episode)\b',' ', cleanr)
#     cleanr = re.sub(r'\b(look forward|looking forward|great work|great job|good job|good work|highly recommend)+(s?|es?)\b',' ', cleanr)
#     cleanr = re.sub(r'\b(great work|great job|look forward|looking forward|good job|good work|great show|listened episode|feel like)\b',' ', cleanr)
#     cleanr = re.sub(r'\b(favorite|absolutely|great|highly recommend|new|episode|love|enjoy|listen|listening|hard work|true crime)\b',' ', cleanr)
    
    word_tokens = word_tokenize(cleanr)
    
    if REMOVE_STOP_WORDS == True:
        filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        word_tokens = filtered_sentence
    
    if STEMMING == True:
        word_tokens = [ps.stem(w) for w in word_tokens]
        
    if LEMMATIZE == True:
        word_tokens = [lemmatizer.lemmatize(w) for w in word_tokens]
        
    output_text = ' '.join(word_tokens)
    return output_text

In [147]:
neg['clean_content'] = neg.apply(lambda x: cleantxt(x['content']), axis=1)

/var/folders/zy/z2gfj07x2s187jy3_c87nhj80000gq/T/ipykernel_2055/2175117729.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg['clean_content'] = neg.apply(lambda x: cleantxt(x['content']), axis=1)


In [148]:
# score again 
vectorizer = TfidfVectorizer(ngram_range=(2,2),
                             token_pattern=r'\b[a-zA-Z]{3,}\b',
                             max_df=0.4, max_features=1000, stop_words=stopwords.words())

corpus = list(neg["clean_content"].values)

X = vectorizer.fit_transform(corpus)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)
tf_idf = tf_idf.sum(axis=1)
score = pd.DataFrame(tf_idf, columns=["score"])
score.sort_values(by="score", ascending=False, inplace=True)
score.head(30)

/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aldiz', 'baiknya', 'baizik', 'berkali', 'bukatzeko', 'edota', 'eze', 'ezpabere', 'ezpada', 'ezperen', 'gainera', 'gainerontzean', 'guztiz', 'hainbestez', 'horra', 'kali', 'kurangnya', 'mata', 'olah', 'ordea', 'osterantzean', 'printr', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(
/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,score
waste time,242.263745
long time,108.694448
stop listening,105.166790
give star,87.640237
started listening,87.280937
sound quality,86.148968
subject matter,82.630875
listened episode,79.688357
bring back,78.889662
stopped listening,75.864706


In [149]:
vectorizer1 = TfidfVectorizer(ngram_range=(2,2),
                             min_df=0.01, max_df=0.6, stop_words="english")


X_review1, review_terms1 = vectorizer1.fit_transform(neg.clean_content), vectorizer1.get_feature_names_out()
review_tf_idf1 = pd.DataFrame(X_review1.toarray(), columns=review_terms1)
print(f"Podcast Review TF-IDF: {review_tf_idf1.shape}")
review_tf_idf1.head(5)

Podcast Review TF-IDF: (21751, 2)


,true crime,waste time
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0


In [150]:
nmf1 = NMF(n_components=3)
W_review1 = nmf1.fit_transform(X_review1)
H_review1 = nmf1.components_
print(f"Original shape of X review is {X_review1.shape}")
print(f"Decomposed W review matrix is {W_review1.shape}")
print(f"Decomposed H review matrix is {H_review1.shape}")

Original shape of X review is (21751, 2)
Decomposed W review matrix is (21751, 3)
Decomposed H review matrix is (3, 2)


/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


In [151]:
from typing import List

def get_top_tf_idf_tokens_for_topic(H: np.array, feature_names: List[str], num_top_tokens: int = 5):
  """
  Uses the H matrix (K components x M original features) to identify for each
  topic the most frequent tokens.
  """
  for topic, vector in enumerate(H):
    print(f"TOPIC {topic}\n")
    total = vector.sum()
    top_scores = vector.argsort()[::-1][:num_top_tokens]
    token_names = list(map(lambda idx: feature_names[idx], top_scores))
    strengths = list(map(lambda idx: vector[idx] / total, top_scores))
    
    for strength, token_name in zip(strengths, token_names):
          print(f"\b{token_name} ({round(strength * 100, 1)}%)\n")
    print(f"=" * 50)

In [152]:
get_top_tf_idf_tokens_for_topic(H_review1, review_tf_idf1.columns.tolist(), 5)
print(f"Podcast Review Topics:\n\n")

TOPIC 0

waste time (100.0%)

true crime (0.0%)

TOPIC 1

true crime (100.0%)

waste time (0.0%)

TOPIC 2

true crime (100.0%)

waste time (0.0%)

Podcast Review Topics:




### negative: news

In [128]:
neg_news = df[(df['rating']<4)&(df['category']=='news')]

In [129]:
vectorizer = TfidfVectorizer(ngram_range=(2,2),
                             token_pattern=r'\b[a-zA-Z]{3,}\b',
                             max_df=0.4, max_features=1000, stop_words=stopwords.words())

corpus = list(neg_news["content"].values)

X = vectorizer.fit_transform(corpus)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)
tf_idf = tf_idf.sum(axis=1)
score = pd.DataFrame(tf_idf, columns=["score"])
score.sort_values(by="score", ascending=False, inplace=True)
score.head(30)

/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aldiz', 'baiknya', 'baizik', 'berkali', 'bukatzeko', 'edota', 'eze', 'ezpabere', 'ezpada', 'ezperen', 'gainera', 'gainerontzean', 'guztiz', 'hainbestez', 'horra', 'kali', 'kurangnya', 'mata', 'olah', 'ordea', 'osterantzean', 'printr', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(
/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,score
waste time,29.478363
listen podcast,22.960849
left wing,17.882479
long time,17.436235
talking points,13.845068
point view,12.828270
love show,12.752025
listen show,12.496078
fox news,12.379474
love podcast,12.232290


In [140]:
def cleantxt(cleanr):
    cleanr = cleanr.strip().lower()
    cleanr = re.sub(r'\b(podcast|show)+(s?|es?)\b',' ', cleanr)
    cleanr = re.sub(r'\b(waste time|feel like|true crime|sound like)\b',' ', cleanr)
    cleanr = re.sub(r'\b(listen|listening|stop|stopped|love)\b',' ', cleanr)
#     cleanr = re.sub(r'\b(love|like|great|good|really|really enojoy|true crime|started listening|listen|listening|new episode)\b',' ', cleanr)
#     cleanr = re.sub(r'\b(look forward|looking forward|great work|great job|good job|good work|highly recommend)+(s?|es?)\b',' ', cleanr)
#     cleanr = re.sub(r'\b(great work|great job|look forward|looking forward|good job|good work|great show|listened episode|feel like)\b',' ', cleanr)
#     cleanr = re.sub(r'\b(favorite|absolutely|great|highly recommend|new|episode|love|enjoy|listen|listening|hard work|true crime)\b',' ', cleanr)
    
    word_tokens = word_tokenize(cleanr)
    
    if REMOVE_STOP_WORDS == True:
        filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        word_tokens = filtered_sentence
    
    if STEMMING == True:
        word_tokens = [ps.stem(w) for w in word_tokens]
        
    if LEMMATIZE == True:
        word_tokens = [lemmatizer.lemmatize(w) for w in word_tokens]
        
    output_text = ' '.join(word_tokens)
    return output_text

In [141]:
neg_news['clean_content'] = neg_news.apply(lambda x: cleantxt(x['content']), axis=1)

/var/folders/zy/z2gfj07x2s187jy3_c87nhj80000gq/T/ipykernel_2055/79664435.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_news['clean_content'] = neg_news.apply(lambda x: cleantxt(x['content']), axis=1)


In [142]:
vectorizer1 = TfidfVectorizer(ngram_range=(2,2),
                             min_df=0.01, max_df=0.4, stop_words="english")


X_review1, review_terms1 = vectorizer1.fit_transform(neg_news.clean_content), vectorizer1.get_feature_names_out()
review_tf_idf1 = pd.DataFrame(X_review1.toarray(), columns=review_terms1)
print(f"Podcast Review TF-IDF: {review_tf_idf1.shape}")
review_tf_idf1.head(5)

Podcast Review TF-IDF: (3202, 4)


,ben shapiro,point view,sound like,waste time
0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0


In [143]:
nmf1 = NMF(n_components=3)
W_review1 = nmf1.fit_transform(X_review1)
H_review1 = nmf1.components_
print(f"Original shape of X review is {X_review1.shape}")
print(f"Decomposed W review matrix is {W_review1.shape}")
print(f"Decomposed H review matrix is {H_review1.shape}")

/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


Original shape of X review is (3202, 4)
Decomposed W review matrix is (3202, 3)
Decomposed H review matrix is (3, 4)


/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [144]:
from typing import List

def get_top_tf_idf_tokens_for_topic(H: np.array, feature_names: List[str], num_top_tokens: int = 5):
  """
  Uses the H matrix (K components x M original features) to identify for each
  topic the most frequent tokens.
  """
  for topic, vector in enumerate(H):
    print(f"TOPIC {topic}\n")
    total = vector.sum()
    top_scores = vector.argsort()[::-1][:num_top_tokens]
    token_names = list(map(lambda idx: feature_names[idx], top_scores))
    strengths = list(map(lambda idx: vector[idx] / total, top_scores))
    
    for strength, token_name in zip(strengths, token_names):
          print(f"\b{token_name} ({round(strength * 100, 1)}%)\n")
    print(f"=" * 50)

In [145]:
get_top_tf_idf_tokens_for_topic(H_review1, review_tf_idf1.columns.tolist(), 5)
print(f"Podcast Review Topics:\n\n")

TOPIC 0

sound like (100.0%)

waste time (0.0%)

point view (0.0%)

ben shapiro (0.0%)

TOPIC 1

waste time (96.0%)

point view (4.0%)

sound like (0.0%)

ben shapiro (0.0%)

TOPIC 2

ben shapiro (100.0%)

waste time (0.0%)

sound like (0.0%)

point view (0.0%)

Podcast Review Topics:




### negative: health

In [188]:
neg_health = df[(df['rating']<4)&(df['category']=='health')]

In [189]:
neg_health['clean_content'] = neg_health.apply(lambda x: cleantxt(x['content']), axis=1)

/var/folders/zy/z2gfj07x2s187jy3_c87nhj80000gq/T/ipykernel_2055/3074378791.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_health['clean_content'] = neg_health.apply(lambda x: cleantxt(x['content']), axis=1)


In [190]:
vectorizer1 = TfidfVectorizer(ngram_range=(2,2),
                             min_df=0.01, max_df=0.4, stop_words="english")


X_review1, review_terms1 = vectorizer1.fit_transform(neg_health.clean_content), vectorizer1.get_feature_names_out()
review_tf_idf1 = pd.DataFrame(X_review1.toarray(), columns=review_terms1)
print(f"Podcast Review TF-IDF: {review_tf_idf1.shape}")
review_tf_idf1.head(5)

Podcast Review TF-IDF: (652, 13)


,audio quality,episode listened,felt like,hard time,listened episode,listening episode,long time,sound like,sound quality,started listening,stop listening,want hear,waste time
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [191]:
nmf1 = NMF(n_components=3)
W_review1 = nmf1.fit_transform(X_review1)
H_review1 = nmf1.components_
print(f"Original shape of X review is {X_review1.shape}")
print(f"Decomposed W review matrix is {W_review1.shape}")
print(f"Decomposed H review matrix is {H_review1.shape}")

Original shape of X review is (652, 13)
Decomposed W review matrix is (652, 3)
Decomposed H review matrix is (3, 13)


/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [192]:
get_top_tf_idf_tokens_for_topic(H_review1, review_tf_idf1.columns.tolist(), 5)
print(f"Podcast Review Topics:\n\n")

TOPIC 0

sound like (96.7%)

sound quality (3.1%)

audio quality (0.2%)

want hear (0.1%)

waste time (0.0%)

TOPIC 1

stop listening (76.8%)

long time (15.0%)

started listening (7.6%)

listening episode (0.6%)

episode listened (0.0%)

TOPIC 2

listened episode (100.0%)

waste time (0.0%)

want hear (0.0%)

stop listening (0.0%)

started listening (0.0%)

Podcast Review Topics:




### negative: science

In [194]:
neg_science = df[(df['rating']<4)&(df['category']=='science')]

In [195]:
neg_science['clean_content'] = neg_science.apply(lambda x: cleantxt(x['content']), axis=1)

/var/folders/zy/z2gfj07x2s187jy3_c87nhj80000gq/T/ipykernel_2055/1949001041.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_science['clean_content'] = neg_science.apply(lambda x: cleantxt(x['content']), axis=1)


In [ ]:
vectorizer1 = TfidfVectorizer(ngram_range=(2,2),
                             min_df=0.01, max_df=0.4, stop_words="english")


X_review1, review_terms1 = vectorizer1.fit_transform(neg_health.clean_content), vectorizer1.get_feature_names_out()
review_tf_idf1 = pd.DataFrame(X_review1.toarray(), columns=review_terms1)
print(f"Podcast Review TF-IDF: {review_tf_idf1.shape}")
review_tf_idf1.head(5)

### Genism & LDA

In [172]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [173]:
# Convert to list 
data = neg.content.values.tolist()  
# Remove Emails 
data = [re.sub('\d(podcast|show)+(s?|es?)\d', '', cleanr) for cleanr in data]  
# Remove new line characters 
data = [re.sub('\d(true crime|waste time|sound like|point view)\d', ' ', cleanr) for cleanr in data]  

pprint(data[:1])

['Last season there was a new pod every 3-4 days when the season started. Now '
 '2 weeks into the season and there are only 2 pods in that time.  Time to '
 'start shopping for a new Phillies podcast.']


<>:4: DeprecationWarning: invalid escape sequence \d
<>:6: DeprecationWarning: invalid escape sequence \d
<>:4: DeprecationWarning: invalid escape sequence \d
<>:6: DeprecationWarning: invalid escape sequence \d
/var/folders/zy/z2gfj07x2s187jy3_c87nhj80000gq/T/ipykernel_2055/3176144363.py:4: DeprecationWarning: invalid escape sequence \d
  data = [re.sub('\d(podcast|show)+(s?|es?)\d', '', cleanr) for cleanr in data]
/var/folders/zy/z2gfj07x2s187jy3_c87nhj80000gq/T/ipykernel_2055/3176144363.py:6: DeprecationWarning: invalid escape sequence \d
  data = [re.sub('\d(true crime|waste time|sound like|point view)\d', ' ', cleanr) for cleanr in data]


In [174]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  #deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['last', 'season', 'there', 'was', 'new', 'pod', 'every', 'days', 'when', 'the', 'season', 'started', 'now', 'weeks', 'into', 'the', 'season', 'and', 'there', 'are', 'only', 'pods', 'in', 'that', 'time', 'time', 'to', 'start', 'shopping', 'for', 'new', 'phillies', 'podcast']]


In [175]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['last', 'season', 'there', 'was', 'new', 'pod', 'every', 'days', 'when', 'the', 'season', 'started', 'now', 'weeks', 'into', 'the', 'season', 'and', 'there', 'are', 'only', 'pods', 'in', 'that', 'time', 'time', 'to', 'start', 'shopping', 'for', 'new', 'phillies', 'podcast']


In [176]:
# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [178]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_md", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['last', 'season', 'new', 'pod', 'day', 'season', 'start', 'week', 'season', 'pod', 'time', 'time', 'start', 'shop', 'new', 'phillie', 'podcast']]


In [179]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 2), (5, 1), (6, 3), (7, 1), (8, 2), (9, 2), (10, 1)]]


In [180]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('day', 1),
  ('last', 1),
  ('new', 2),
  ('phillie', 1),
  ('pod', 2),
  ('podcast', 1),
  ('season', 3),
  ('shop', 1),
  ('start', 2),
  ('time', 2),
  ('week', 1)]]

In [181]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [182]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.077*"show" + 0.035*"host" + 0.019*"new" + 0.017*"see" + 0.014*"interview" '
  '+ 0.010*"issue" + 0.010*"old" + 0.009*"become" + 0.008*"season" + '
  '0.008*"far"'),
 (1,
  '0.049*"podcast" + 0.033*"listen" + 0.024*"get" + 0.016*"time" + '
  '0.016*"make" + 0.014*"talk" + 0.014*"really" + 0.014*"good" + 0.014*"say" + '
  '0.014*"go"'),
 (2,
  '0.018*"people" + 0.014*"life" + 0.014*"right" + 0.012*"question" + '
  '0.011*"believe" + 0.009*"fun" + 0.009*"learn" + 0.008*"help" + '
  '0.008*"world" + 0.008*"ask"'),
 (3,
  '0.036*"woman" + 0.032*"man" + 0.017*"audio" + 0.016*"trump" + 0.014*"movie" '
  '+ 0.012*"kid" + 0.011*"quite" + 0.011*"entire" + 0.009*"claim" + '
  '0.009*"medium"'),
 (4,
  '0.075*"episode" + 0.028*"guest" + 0.021*"first" + 0.018*"minute" + '
  '0.017*"come" + 0.016*"topic" + 0.015*"interesting" + 0.015*"year" + '
  '0.015*"back" + 0.014*"ad"')]


In [184]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/en-ning.chiang/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/User

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.414969  0.014703       1        1  50.465380
4     -0.082522 -0.352705       2        1  16.084382
0     -0.102546  0.248776       3        1  15.259310
2     -0.100107  0.063493       4        1  12.511046
3     -0.129794  0.025733       5        1   5.679882, topic_info=         Term          Freq         Total Category  logprob  loglift
128      show   5847.000000   5847.000000  Default  30.0000  30.0000
28    episode   5992.000000   5992.000000  Default  29.0000  29.0000
5     podcast  12335.000000  12335.000000  Default  28.0000  28.0000
65     listen   8213.000000   8213.000000  Default  27.0000  27.0000
164      host   3028.000000   3028.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
1341   suffer    163.894198    164.732018   Topic5  -5.1525   2.8631
1324     line    160.857607    161.680370   Topic5  -5.1712   2.8631
795      form    154.738634    155.564032   Topic5  -5.2100   2.8629
1248  helpful    153.469810    154.298355   Topic5  -5.2182   2.8629
193    agenda    152.874033    153.704355   Topic5  -5.2221   2.8628

[196 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
832       3  0.997160        able
151       3  0.998296  absolutely
1401      4  0.997909         act
45        2  0.999699          ad
193       5  0.995417      agenda
...     ...       ...         ...
230       4  0.997861       world
177       3  0.130204       wrong
177       4  0.869247       wrong
668       2  0.999597        year
1309      5  0.998060         yet

[181 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 1, 3, 4])